# Decorators
This notebook covers tips on decorator and particularly on how to integrate them into class definition.

### 1. Recall decorator definition

#### a. Simple decorator

In [4]:
def my_func(*args):
    print(*args)

In [5]:
def decorator(func):
    def inner(*args):
        print("has been decorated")
        return func(*args)
    return inner

In [6]:
decorated_func = decorator(my_func)
decorated_func # return a inner function which takes *args as argument

<function __main__.decorator.<locals>.inner(*args)>

In [7]:
decorated_func(1,2,3)

has been decorated
1 2 3


In [8]:
# using syntactic sugar
@decorator
def my_func(*args):
    print(*args)

In [9]:
my_func(1,2)

has been decorated
1 2


#### b. Decorator factory

In [11]:
def my_func(*args):
    print(*args)

In [12]:
def decorator_with_param(param):
    def decorator(func):
        def inner(*args):
            print(f"has been decorated with {param}")
            return func(*args)
        return inner
    return decorator

In [13]:
decorator = decorator_with_param("my param")
decorator # return a decorator function

<function __main__.decorator_with_param.<locals>.decorator(func)>

In [14]:
decorated_func = decorator(my_func)
decorated_func # return inner function which in turns can take *args as arguments

<function __main__.decorator_with_param.<locals>.decorator.<locals>.inner(*args)>

In [15]:
decorated_func(1,2,3,4,5)

has been decorated with my param
1 2 3 4 5


In [16]:
# using syntactic sugar
@decorator_with_param("my param")
def my_func(*args):
    print(*args)

In [17]:
my_func(1,2,3,4,5,6)

has been decorated with my param
1 2 3 4 5 6


### 2. Integrate decorator within classes

In [19]:
# decorator can also be called outside the class scope
def outside_decorator(func):
    def inner(my_class_object, *args):
        print("outside decorator", (my_class_object.value))
        func(my_class_object, *args)
    return inner

In [20]:
class MyClass:

    def __init__(self, value):
        self.value = value

    def decorator(func):
        def inner(self, *args):
            print("decorated", self.value)
            func(self, *args)
        return inner

    def get_value(self):
        print(self.value)

    # decorator can be called within the class space because no ref call like a.decorator
    decorated_func = decorator(get_value)

    @decorator
    def get_value2(self):
        print(self.value)

    @outside_decorator
    def get_value3(self):
        print(self.value)

    def test():
        print('test')

    @staticmethod
    def test2():
        print('test')

In [21]:
a = MyClass("a")

In [22]:
a.get_value()

a


In [23]:
a.decorated_func()

decorated a
a


In [24]:
a.get_value2()

decorated a
a


In [25]:
a.get_value3()

outside decorator a
a


In [26]:
a.__dict__

{'value': 'a'}

In [27]:
MyClass.__dict__ # decorator is in the class namesapce

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.MyClass.__init__(self, value)>,
              'decorator': <function __main__.MyClass.decorator(func)>,
              'get_value': <function __main__.MyClass.get_value(self)>,
              'decorated_func': <function __main__.MyClass.decorator.<locals>.inner(self, *args)>,
              'get_value2': <function __main__.MyClass.decorator.<locals>.inner(self, *args)>,
              'get_value3': <function __main__.outside_decorator.<locals>.inner(my_class_object, *args)>,
              'test': <function __main__.MyClass.test()>,
              'test2': <staticmethod(<function MyClass.test2 at 0x000001A5DA091580>)>,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [28]:
MyClass.decorator

<function __main__.MyClass.decorator(func)>

In [29]:
MyClass.get_value

<function __main__.MyClass.get_value(self)>

In [30]:
# A function within the class name space without ref to self cannot be called
# Indeed when calling a.test() Python automatically pass a to test -> test(a) 
# But if there is no args in test signature then there will be a missing argument
try:
    t = a.test
    t()
except TypeError as err:
    print(err)

MyClass.test() takes 0 positional arguments but 1 was given


In [31]:
# however with the @staticmethod decorator, that will work because a is not passed as argument
t = a.test2
t()

test


In [32]:
# However test can be called from MyClass because 
MyClass.test()

test


In [33]:
# func can be decorated with a regular decorator
b = MyClass("b")

@outside_decorator
def func(obj, *args):
    print(obj.value, args)

func(b, 1,2,3)

outside decorator b
b (1, 2, 3)


In [34]:
# it can also be decorated with deco which is retrieved from MyClass
deco = MyClass.decorator
deco

<function __main__.MyClass.decorator(func)>

In [35]:
@deco
def func(obj, *args):
    print(obj.value, args)

func(b, 1,2,3)

decorated b
b (1, 2, 3)
